In [110]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import json
# import PyPDF2

In [85]:
# !pip install langchain_huggingface
# !pip install transformers


In [111]:
from langchain_huggingface import HuggingFaceEndpoint
# from langchain_community.llms import HuggingFaceEndpoint

In [112]:
# from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.5,
    huggingfacehub_api_token="Enter Your Hugging Face Access Token Here",
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\PMLS\.cache\huggingface\token
Login successful


In [113]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [114]:
TEXT=""""Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, biologists use the scientific method to make observations, pose questions, generate hypotheses, perform experiments, and form conclusions about the world around them.[1]

Life on Earth, which emerged more than 3.7 billion years ago,[10] is immensely diverse. Biologists have sought to study and classify the various forms of life, from prokaryotic organisms such as archaea and bacteria to eukaryotic organisms such as protists, fungi, plants, and animals. These various organisms contribute to the biodiversity of an ecosystem, where they play specialized roles in the cycling of nutrients and energy through their biophysical environment."""

In [115]:
NUMBER=5 
SUBJECT="biology"
TONE="hard"

In [116]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### The below is RESPONSE_JSON
{response_json}

"""

In [117]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [118]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [119]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [120]:

quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"], 
    template=TEMPLATE2
    )

In [121]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [122]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [123]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:"Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defin

In [124]:
# print(f"Total Tokens:{cb.total_tokens}")
# print(f"Prompt Tokens:{cb.prompt_tokens}")
# print(f"Completion Tokens:{cb.completion_tokens}")
# print(f"Total Cost:{cb.total_cost}")

In [125]:
response

{'text': '"Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sc

In [126]:
quiz=response.get("quiz")

In [127]:
print(quiz)

### RESPONSE_JSON
{
"1": {
    "mcq": "What is the scientific study of life?",
    "options": {
        "a": "Physics",
        "b": "Chemistry",
        "c": "Biology",
        "d": "Mathematics"
    },
    "correct": "c"
},
"2": {
    "mcq": "What is a major theme in biology that explains the unity and diversity of life?",
    "options": {
        "a": "Evolution",
        "b": "Cell division",
        "c": "Photosynthesis",
        "d": "DNA replication"
    },
    "correct": "a"
},
"3": {
    "mcq": "What is the process that allows organisms to move, grow, and reproduce?",
    "options": {
        "a": "Respiration",
        "b": "Energy processing",
        "c": "Photosynthesis",
        "d": "Cell division"
    },
    "correct": "b"
},
"4": {
    "mcq": "What is the study of the various forms of life and their classification?",
    "options": {
        "a": "Taxonomy",
        "b": "Physics",
        "c": "Chemistry",
        "d": "Mathematics"
    },
    "correct": "a"
},
"5": {

In [128]:
quiz

'### RESPONSE_JSON\n{\n"1": {\n    "mcq": "What is the scientific study of life?",\n    "options": {\n        "a": "Physics",\n        "b": "Chemistry",\n        "c": "Biology",\n        "d": "Mathematics"\n    },\n    "correct": "c"\n},\n"2": {\n    "mcq": "What is a major theme in biology that explains the unity and diversity of life?",\n    "options": {\n        "a": "Evolution",\n        "b": "Cell division",\n        "c": "Photosynthesis",\n        "d": "DNA replication"\n    },\n    "correct": "a"\n},\n"3": {\n    "mcq": "What is the process that allows organisms to move, grow, and reproduce?",\n    "options": {\n        "a": "Respiration",\n        "b": "Energy processing",\n        "c": "Photosynthesis",\n        "d": "Cell division"\n    },\n    "correct": "b"\n},\n"4": {\n    "mcq": "What is the study of the various forms of life and their classification?",\n    "options": {\n        "a": "Taxonomy",\n        "b": "Physics",\n        "c": "Chemistry",\n        "d": "Mathemati

In [129]:
import json

json_str = quiz.replace('### RESPONSE_JSON\n', '')

# Parse the JSON string
json_quiz_responce = json.loads(json_str)

In [130]:
json_quiz_responce['1']

{'mcq': 'What is the scientific study of life?',
 'options': {'a': 'Physics',
  'b': 'Chemistry',
  'c': 'Biology',
  'd': 'Mathematics'},
 'correct': 'c'}

In [106]:
for i in range(len(json_quiz_responce)):
    index=str(i+1)
    print(json_quiz_responce[index]['mcq'])

What is the scientific study of life?
What is a major theme in biology that explains the unity and diversity of life?
What is the process that allows organisms to move, grow, and reproduce?
What is the study of the various forms of life and their classification?
What is the role of organisms in an ecosystem?


In [107]:
for i in range(len(json_quiz_responce)):
    index=str(i+1)
    print(json_quiz_responce[index]['correct'])

c
a
b
a
a
